In [7]:
from qdrant_client import QdrantClient, models


In [8]:
model_handle = "jinaai/jina-embeddings-v2-small-en"


In [9]:
text = 'I just discovered the course. Can I join now?'

In [12]:
client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance

In [10]:
vector=models.Document(text=text, model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed

In [26]:
from fastembed import TextEmbedding
import numpy as np

# Cargar el modelo Jina
model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

# Texto a embedir
query = "I just discovered the course. Can I join now?"

# Obtener el embedding (es un generador, así que usamos next)
q = next(model.embed([query]))




In [27]:
q.min()

np.float64(-0.11726373885183883)

In [28]:
import numpy as np
np.linalg.norm(q)

np.float64(1.0)

In [29]:
q.dot(q)

np.float64(1.0000000000000002)

In [30]:
doc = 'Can I still join the course after the start date?'
# Obtener el embedding (es un generador, así que usamos next)
doc_emb = next(model.embed([doc]))


In [36]:
print(q)
print("=============================")
print(doc_emb)


[-7.63946260e-02 -7.30555505e-02  5.86501763e-02  3.92670571e-02
 -1.41797115e-02 -4.68342854e-02  2.69862822e-02  3.49039060e-02
  1.53419217e-03 -1.10407363e-02  5.57556285e-02 -2.88360142e-02
 -4.87239498e-02 -9.90923923e-02  6.02579764e-02 -1.86105379e-02
 -8.81040139e-03 -3.01994831e-02 -1.42541815e-02 -3.87884212e-02
 -2.76125662e-02 -6.94957533e-03  3.20653319e-02 -5.17838394e-03
  8.37959894e-02 -8.87329606e-02 -7.30260625e-02  5.92846802e-02
  4.07805674e-02  7.72684822e-02 -4.60029697e-02  3.98426895e-02
  1.74821510e-02  8.70263712e-03 -3.09291054e-02  2.20470391e-02
  4.76479896e-02  1.76566196e-02 -3.62013627e-02 -3.53408062e-02
 -5.92537995e-03  2.97091202e-02  8.05883752e-02  1.07132722e-02
 -5.38192519e-02 -9.62881111e-03 -1.17263739e-01  3.36546374e-02
  7.05763168e-03  7.70159791e-03 -5.17482529e-02  5.46955574e-02
 -7.80289904e-02  4.21801655e-02  1.26250977e-02  5.31339061e-02
 -2.67777188e-02  1.68820697e-02 -3.75064593e-02  9.13196419e-03
  2.13696828e-02  1.60150

In [37]:



cosine_similarity = np.dot(q, doc_emb) / (np.linalg.norm(q) * np.linalg.norm(doc_emb))

print("Similitud del coseno:", cosine_similarity)


Similitud del coseno: 0.9008528895674548


In [39]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [43]:
doc_text_array =[]
for doc in documents:
    

    vector_tex = next(model.embed(doc['text']))
    doc_text_array.append(vector_tex)

In [47]:
import numpy as np

# Supongamos que q ya es un numpy array
q = np.array(q)  # solo si aún no lo es

# Convertir la lista de vectores a un array 2D
doc_matrix = np.array(doc_text_array)  # shape: (n_docs, dim)

# Calcular similitudes de coseno para todos los vectores
cos_similarities = np.dot(doc_matrix, q) / (np.linalg.norm(doc_matrix, axis=1) * np.linalg.norm(q))

print(cos_similarities)  # array de similitudes, uno por documento


[0.76296847 0.81823782 0.80853974 0.7133079  0.73044992]


In [48]:
cos_similarities.max()

np.float64(0.8182378150042889)

In [49]:

doc_text_array_v2 =[]
for doc in documents:
    full_text = doc['question'] + ' ' + doc['text']
    vector_tex = next(model.embed(full_text))
    doc_text_array_v2.append(vector_tex)

In [50]:
doc_text_array_v2

[array([-3.95874826e-02, -6.90406894e-02,  1.14661797e-02,  2.62603988e-02,
        -2.47341027e-02, -2.47207056e-02,  3.43971205e-03,  4.27859250e-02,
        -2.41629394e-02, -2.61992357e-02, -1.49214508e-02, -7.88178399e-02,
        -2.47400895e-02, -7.74014517e-02,  7.68642757e-02, -4.65683749e-02,
        -3.08454460e-02, -4.45657217e-02, -4.48141420e-02, -6.06556355e-02,
        -3.67113226e-02, -1.80009404e-02,  3.19522744e-02, -6.80573414e-02,
         5.93400861e-02, -7.10708989e-02, -8.38071366e-02,  7.13435390e-02,
         7.23555512e-02,  2.43478195e-02, -5.51024677e-02,  2.94591679e-02,
        -1.83082436e-02,  2.56099754e-02, -3.17286899e-02,  2.76751097e-02,
         4.96358524e-02,  1.38839466e-02, -2.01003038e-02, -2.82512659e-03,
        -2.93719094e-02,  2.82632724e-02,  1.00648435e-01,  2.89436669e-03,
        -7.16561258e-02,  7.67853418e-04, -8.02424389e-02,  1.12407843e-02,
         2.55503821e-02,  1.53881720e-02, -3.34380302e-02,  2.85950294e-02,
        -7.9

In [51]:
# Convertir la lista de vectores a un array 2D
doc_matrix_v2 = np.array(doc_text_array_v2)  # shape: (n_docs, dim)

# Calcular similitudes de coseno para todos los vectores
cos_similarities_2 = np.dot(doc_matrix_v2, q) / (np.linalg.norm(doc_matrix_v2, axis=1) * np.linalg.norm(q))

print(cos_similarities_2)  # array de similitudes, uno por documento

[0.85145432 0.84365942 0.8408287  0.7755158  0.80860078]


In [ ]:
small_model = "BAAI/bge-small-en"


# Cargar el modelo Jina
model = TextEmbedding(small_model)

# Texto a embedir
query = "I just discovered the course. Can I join now?"

# Obtener el embedding (es un generador, así que usamos next)
q1 = next(model.embed([query]))


Fetching 5 files: 100%|██████████| 5/5 [00:07<00:00,  1.48s/it]


In [53]:
len(q1)

384

In [54]:
# con quadrant

In [56]:
from qdrant_client import QdrantClient, models


In [55]:
client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance


In [57]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [58]:
EMBEDDING_DIMENSIONALITY = 384


In [60]:
model_handle = "BAAI/bge-small-en"

In [61]:
# Define the collection name
collection_name = "homework_2-sc"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [62]:
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp'}

In [63]:
points = []
id = 0

for course in documents_raw:
    for doc in course['documents']:
        text = doc['question'] + ' ' + doc['text']
        point = models.PointStruct(
            id=id,
            vector=models.Document(text=text, model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
            payload={
                "text": doc['text'],
                "section": doc['section'],
                "course": course['course']
            } #save all needed metadata fields
        )
        points.append(point)

        id += 1

In [64]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [65]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [ ]:
query = "I just discovered the course. Can I join now?"

result = search(query)


In [67]:
result

QueryResponse(points=[ScoredPoint(id=449, version=0, score=0.8703172, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)])